In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./dataset.csv')

In [3]:
df["CRASH_DATE"] = pd.to_datetime(df["CRASH_DATE"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")

In [4]:
columns_to_keep = [
    'CRASH_RECORD_ID', 'CRASH_DATE', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH',
    'WEATHER_CONDITION', 'ROADWAY_SURFACE_COND',
    'POSTED_SPEED_LIMIT', 'LANE_CNT'
]
df = df[columns_to_keep]

In [5]:
df["WEATHER_CONDITION"].value_counts()

WEATHER_CONDITION
CLEAR                       588571
RAIN                         64056
UNKNOWN                      39154
SNOW                         26438
CLOUDY/OVERCAST              21994
OTHER                         2338
FREEZING RAIN/DRIZZLE         1368
FOG/SMOKE/HAZE                1100
SLEET/HAIL                     934
BLOWING SNOW                   392
SEVERE CROSS WIND GATE         147
BLOWING SAND, SOIL, DIRT         6
Name: count, dtype: int64

In [6]:
weather_mapping = {
    "CLEAR": "CLEAR",
    "RAIN": "RAIN",
    "FREEZING RAIN/DRIZZLE": "RAIN",
    "SLEET/HAIL": "RAIN",
    "SNOW": "SNOW",
    "BLOWING SNOW": "SNOW",
    "FOG/SMOKE/HAZE": "CLEAR",  
    "CLOUDY/OVERCAST": "CLEAR",
    "SEVERE CROSS WIND GATE": "CLEAR",
    "BLOWING SAND, SOIL, DIRT": "OTHER",
    "UNKNOWN": "OTHER",
    "OTHER": "OTHER"
}

df["WEATHER_CONDITION"] = df["WEATHER_CONDITION"].map(weather_mapping)

In [7]:
df["ROADWAY_SURFACE_COND"].value_counts()

ROADWAY_SURFACE_COND
DRY                554470
WET                 98336
UNKNOWN             59857
SNOW OR SLUSH       26458
ICE                  5214
OTHER                1877
SAND, MUD, DIRT       286
Name: count, dtype: int64

In [8]:
road_condition_mapping = {
    "DRY": "DRY",
    "WET": "WET",
    "UNKNOWN": "UNKNOWN",
    "SNOW OR SLUSH": "SNOW",
    "ICE": "ICE",
    "OTHER": "OTHER",
    "SAND, MUD, DIRT": "OTHER"
}
df["ROADWAY_SURFACE_COND"] = df["ROADWAY_SURFACE_COND"].map(road_condition_mapping)

In [9]:
df["ROADWAY_SURFACE_COND"].value_counts()

ROADWAY_SURFACE_COND
DRY        554470
WET         98336
UNKNOWN     59857
SNOW        26458
ICE          5214
OTHER        2163
Name: count, dtype: int64

In [10]:
# Proportionally distribute UNKNOWN road condition
known_counts = df[df["ROADWAY_SURFACE_COND"] != "UNKNOWN"]["ROADWAY_SURFACE_COND"].value_counts(normalize=True)

unknown_mask = df["ROADWAY_SURFACE_COND"] == "UNKNOWN"
df.loc[unknown_mask, "ROADWAY_SURFACE_COND"] = np.random.choice(known_counts.index, size=unknown_mask.sum(), p=known_counts.values)

In [11]:
df["ROADWAY_SURFACE_COND"].value_counts()

ROADWAY_SURFACE_COND
DRY      602685
WET      107028
SNOW      28784
ICE        5664
OTHER      2337
Name: count, dtype: int64

In [12]:
# gathering info on the years in the dataset
df['CRASH_YEAR'] = df['CRASH_DATE'].dt.year
year_counts = df['CRASH_YEAR'].value_counts().sort_index()
print(year_counts)

CRASH_YEAR
2013         2
2014         6
2015      9828
2016     44297
2017     83786
2018    118950
2019    117762
2020     92092
2021    108762
2022    108396
2023     62617
Name: count, dtype: int64


In [ ]:
valid_years = year_counts[year_counts >= 65000].index
df = df[df['CRASH_YEAR'].isin(valid_years)]

# 65000 was chosen as a simple way to select the valid years. since all valid years were over this amount anyways.

In [14]:
# Dropping the timestamp from the date for better aggregation
df['CRASH_DATE'] = df['CRASH_DATE'].dt.date

In [15]:
# check for any NaN
print(df.isna().sum())

CRASH_RECORD_ID              0
CRASH_DATE                   0
CRASH_HOUR                   0
CRASH_DAY_OF_WEEK            0
CRASH_MONTH                  0
WEATHER_CONDITION            0
ROADWAY_SURFACE_COND         0
POSTED_SPEED_LIMIT           0
LANE_CNT                476053
CRASH_YEAR                   0
dtype: int64


In [16]:
print(df["LANE_CNT"].value_counts())

LANE_CNT
2.0          71371
4.0          38225
1.0          25071
3.0           6623
0.0           5603
6.0           3268
5.0           1548
8.0           1475
7.0            148
10.0           128
9.0             62
99.0            57
11.0            29
12.0            25
22.0            10
20.0             9
16.0             6
15.0             5
14.0             4
30.0             4
60.0             3
40.0             2
25.0             2
21.0             2
41.0             1
433634.0         1
28.0             1
13.0             1
1191625.0        1
400.0            1
19.0             1
902.0            1
100.0            1
17.0             1
299679.0         1
45.0             1
218474.0         1
24.0             1
80.0             1
Name: count, dtype: int64


In [17]:
daily_crashes = df.groupby('CRASH_DATE').size().reset_index(name='CRASHES_PER_DAY')

In [27]:
daily_crashes

,CRASH_DATE,CRASHES_PER_DAY
0,2017-01-01,138
1,2017-01-02,113
2,2017-01-03,139
3,2017-01-04,103
4,2017-01-05,126
...,...,...
2186,2022-12-27,289
2187,2022-12-28,217
2188,2022-12-29,273
2189,2022-12-30,270


In [18]:
# Coercing these columns to be numeric type, then filling NaNs with the mean values.
df['POSTED_SPEED_LIMIT'] = pd.to_numeric(df['POSTED_SPEED_LIMIT'], errors='coerce')

df.fillna({'POSTED_SPEED_LIMIT': df['POSTED_SPEED_LIMIT'].mean()}, inplace=True)

In [19]:

# Set everything over 10 to be NaN so it's not included in mean caluclation
df.loc[df['LANE_CNT'] > 10, 'LANE_CNT'] = pd.NA
# calculate the mean
lane_cnt_mean = df['LANE_CNT'].mean()
# Fill NaN with the mean
df.fillna({'LANE_CNT': lane_cnt_mean}, inplace=True)

In [20]:
print(df["LANE_CNT"].value_counts())

LANE_CNT
2.492177     476226
2.000000      71371
4.000000      38225
1.000000      25071
3.000000       6623
0.000000       5603
6.000000       3268
5.000000       1548
8.000000       1475
7.000000        148
10.000000       128
9.000000         62
Name: count, dtype: int64


 Some Speed Limits are unrealistic. We need to clean that data.  
Set the valid range between 5-75 since those are the common speed limits seen across the US.
Other unrealistic values in the dataset include speeds like 9 and 44. These are assumed to be typos, so we can fix those by rounding to the nearest 5. Since all speed limits are multiples of 5; 9 and 44 would become 10 and 45 respectively.

In [21]:
print(df['POSTED_SPEED_LIMIT'].unique())

[30 25 20 35 15 10 40 45  5  0 55 50  9 65 24  3  6 34 11 39 60 23 22 14
  1 12 32  2 33  7 26  8 49 36 70 29 63 16 62 44  4 31 38]


In [22]:
valid_speed_limits = range(5, 75)
def round_to_nearest_5(x):
    return 5 * round(x/5)

df['POSTED_SPEED_LIMIT'] = pd.to_numeric(df['POSTED_SPEED_LIMIT'], errors='coerce')
median_speed_limit = df.loc[df['POSTED_SPEED_LIMIT'].between(5, 75), 'POSTED_SPEED_LIMIT'].median()
df.loc[~df['POSTED_SPEED_LIMIT'].between(5, 75), 'POSTED_SPEED_LIMIT'] = median_speed_limit
df['POSTED_SPEED_LIMIT'] = df['POSTED_SPEED_LIMIT'].apply(round_to_nearest_5)
print(df['POSTED_SPEED_LIMIT'].unique())


[30 25 20 35 15 10 40 45  5 55 50 65 60 70]


In [23]:
# Count crashes per day
daily_crash_counts = df.groupby("CRASH_DATE").size().reset_index(name="CRASH_COUNT")

# Merge crash counts back to each record
df = df.merge(daily_crash_counts, on="CRASH_DATE", how="left")

In [24]:
# Verify no NaN left in dataset
print(df.isna().sum())

CRASH_RECORD_ID         0
CRASH_DATE              0
CRASH_HOUR              0
CRASH_DAY_OF_WEEK       0
CRASH_MONTH             0
WEATHER_CONDITION       0
ROADWAY_SURFACE_COND    0
POSTED_SPEED_LIMIT      0
LANE_CNT                0
CRASH_YEAR              0
CRASH_COUNT             0
dtype: int64


In [25]:
# export the final cleaned data to be used for prediction.
final_dataset = df
final_dataset.to_csv("./cleaned_dataset.csv", index=False)